# Face fill

## Imports

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import trimesh

Holistic = mp.solutions.holistic.Holistic

## Helper function

In [2]:
def run_filter_with_mediapipe_model(mediapipe_model, mediapipe_based_filter):
    cap = cv2.VideoCapture(0)

    with mediapipe_model as model:
        while cap.isOpened():
            success, image = cap.read()

            if not success:
                print("Ignoring empty camera frame.")
                continue  # If loading a video, use 'break' instead of 'continue'.

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

            results = model.process(image)

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            result_image = mediapipe_based_filter(image, results)

            cv2.imshow("MediaPipe", result_image)

            if cv2.waitKey(5) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()

## Import canonical mesh

In [3]:
with open("canonical_face_model.obj","r") as objfile:
    scene = trimesh.exchange.obj.load_obj(objfile)
triangles = scene["faces"]

Previous:

```python
triangles = []
with open("canonical_face_model.obj", "r") as wavefile:
    for line in wavefile:
        if line[:1] == "f":
            triangles.append(
                np.array([tup.split("/")[0] for tup in line.split()[1:]], dtype=int) - 1
            )
```

## Pick colors

In [4]:
color = [tuple(np.random.choice(range(256), size=3)) for _ in range(30)]

## Draw function

Previous:
```python
from scipy.spatial import Delaunay

def draw_face(image, results, show_face=True):
    height, width = image.shape[:2]
    if show_face:
        if results.face_landmarks:
            face_points = np.array([[lm.x, lm.y] for lm in results.face_landmarks.landmark])
            tri = Delaunay(face_points)
            for i,triangle in enumerate(tri.simplices):
                triangle_cnt = face_points[triangle]
                triangle_cnt[:,0] *= width
                triangle_cnt[:,1] *= height
                triangle_cnt = triangle_cnt.astype(int)
                cv2.drawContours(image, [triangle_cnt], 0, np.array(color[i%30]).astype(float), -1)

    return image
```

In [5]:
# image_copy = np.zeros((480, 640, 3), dtype=np.uint8)
image_copy = None

def draw_face(image, results, show_face=True):
    global image_copy
    height, width = image.shape[:2]
    if show_face:
        if results.face_landmarks:
            face_points = np.array(
                [[lm.x, lm.y] for lm in results.face_landmarks.landmark]
            )
            if image_copy is None:
                image_copy = np.zeros_like(image)
            image_copy[:, :, :] = image
            for i, triangle in enumerate(triangles):
                triangle_cnt = face_points[triangle]
                triangle_cnt[:, 0] *= width
                triangle_cnt[:, 1] *= height
                triangle_cnt = triangle_cnt.astype(int)
                cv2.drawContours(
                    image_copy,
                    [triangle_cnt],
                    0,
                    np.array(color[i % 30]).astype(float),
                    -1,
                )

            alpha = 0.7
            image = cv2.addWeighted(image, alpha, image_copy, 1 - alpha, 0)

    return image

## Run

In [6]:
run_filter_with_mediapipe_model(
    mediapipe_model=Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5),
    mediapipe_based_filter=draw_face,
)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
